## Define synapomorphies from alignment, for different taxonomic levels

In [4]:
import numpy as np
import pandas as pd
import sqlite3
from __future__ import division
from Bio import AlignIO

In [1]:
%load?

### Start with synapomizer script had built before

In [6]:
# %load -r 22-32 /Users/hughcross/scripts/synapomizer.py
def number_alleles(column):
    leng = len(column)
    bp_list = []
    for bp in range(0,leng):
        pos = column[bp]
        if pos in bp_list:
            pass
        else:
            bp_list.append(pos)
    #no_alleles = len(bp_list) # maybe take out if a gap
    return bp_list

In [25]:
# %load -r 34-49 /Users/hughcross/scripts/synapomizer.py
def informative_alleles(character_dict):
    """a function to filter out uninformative positions"""
    new_dict = {}
    allele_list = []
    for k,v in character_dict.iteritems():
        newd = v
        for key,value in newd.iteritems():
            new_dict.setdefault(key, []).append(value)
    #print new_dict
    for keys, values in new_dict.iteritems():
        alleles = set(values)
        #print keys
        #print alleles
        if len(alleles) > 1:
            allele_list.append(keys)
    return allele_list

In [12]:
# start with a file, then import muscle to make other alignments
# %load -r 9-12 /Users/hughcross/scripts/synapomizer.py
align_file = '/Users/hughcross/analysis/char_trial/partial_mammal_practice1_aln.fasta'
alignment = AlignIO.read(align_file, 'fasta')
length = alignment.get_alignment_length()

In [20]:
# %load -r 14-19 /Users/hughcross/scripts/synapomizer.py
#create a dictionary with sample# : genus name ## note: need to make this part flexible so that different taxonomic levels can be used
name_map = {} # also maybe make a list of each genus, and a dict where key = genus, values = number of species in genus
for i, record in enumerate(alignment):
    fullname = record.id
    newname = fullname.split('_')[0]
    name_map[i]=newname

In [21]:
# %load -r 51-66 /Users/hughcross/scripts/synapomizer.py
pos_list = []
for num in range(0,length):
    position = alignment[:,num] # add num to list
    #print position
    allele_list = number_alleles(position)
    no_alleles = len(allele_list)
    
    if '-' in allele_list:
        no_alleles = no_alleles - 1
    #print no_alleles

    if no_alleles > 1:
        #print position
        pos_list.append(num)
print pos_list
print len(pos_list)

[4, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 47, 52, 53, 54, 55, 57, 59, 62, 63, 64, 65, 66, 68, 79, 83, 84, 87, 94, 96, 99, 100]
49


In [34]:
ambigs = {'AT':'W','TA':'W','CG':'S','GC':'S','CT':'Y','TC':'Y','AG':'R','GA':'R','AC':'M','CA':'M','GT':'K','TG':'K'}

In [37]:
# %load -r 68-85 /Users/hughcross/scripts/synapomizer.py
## note: fixing ambiguous characters in original below (was 'continue')
final_chars = {}

for pos in pos_list:
    gen_char_dict = {}
    for sample in range(0,44):
        char = alignment[sample,pos]
        genus = name_map[sample]
        gen_char_dict.setdefault(genus, []).append(char)
    for key, values in gen_char_dict.iteritems():
        #print key
        newlist = gen_char_dict[key]
        #print newlist
        check = set(newlist)
        #print check
        size = len(check)
        if size > 2:
            #final_chars.setdefault(key, {})[pos]='N'
            newchar = 'N'
        elif size == 2:
            amb_str = ''
            for bp in check:
                amb_str = amb_str + bp
            if '-' in amb_str:
                newchar = amb_str.replace('-','') # may want to count the gaps and adjust later
            else:
                newchar = ambigs[amb_str]
                #final_chars.setdefault(key, {})[pos]=newchar
            
        else:
            newchar = newlist[0]
        final_chars.setdefault(key, {})[pos]=newchar

In [38]:
# %load -r 89-95 /Users/hughcross/scripts/synapomizer.py
# To filter out the characters that have only one alleles; it is variable but only one synapomorphy
## perhaps compare dictionaries in final_chars, if all at one position the same, then delete. 
### try to make it a function 

inform_alleles = informative_alleles(final_chars)
print inform_alleles
print len(inform_alleles)

[4, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 47, 52, 53, 54, 55, 57, 59, 62, 63, 64, 65, 66, 68, 79, 83, 84, 87, 94, 96, 99, 100]
49


In [39]:
# %load -r 97-109 /Users/hughcross/scripts/synapomizer.py
# the informative_alleles function makes a list of informative alleles, then the next loop creates a filtered dictionary of dicts for each genus
filtered_chars1 = {}
for k,v in final_chars.iteritems():
    tax_dict = v  #final_chars[k]
    #print tax_dict
    for key, value in tax_dict.iteritems():
        if key in inform_alleles:
            filtered_chars1.setdefault(k, {})[key]=value

# now maybe sort the dictionary
print filtered_chars1
print name_map
#now to deal with gaps?, maybe later

{'Felis': {4: 'C', 10: 'T', 11: 'T', 12: 'A', 13: 'G', 14: 'A', 15: 'T', 17: 'A', 18: 'G', 19: 'T', 20: 'T', 21: 'A', 22: 'Y', 23: 'C', 24: 'C', 25: 'T', 26: 'A', 27: 'R', 32: 'A', 33: 'A', 34: 'C', 35: 'T', 36: 'A', 37: 'T', 38: 'C', 39: 'C', 40: 'G', 43: 'A', 47: 'A', 52: 'C', 53: 'Y', 54: 'A', 55: 'G', 57: 'A', 59: 'Y', 62: 'C', 63: 'T', 64: 'T', 65: 'A', 66: 'R', 68: 'A', 79: 'T', 83: 'C', 84: 'G', 87: 'G', 94: 'C', 96: 'T', 99: 'C', 100: 'T'}, 'Crocidura': {4: 'C', 10: 'C', 11: 'A', 12: 'A', 13: 'A', 14: 'A', 15: 'C', 17: 'A', 18: 'A', 19: 'T', 20: 'T', 21: 'W', 22: 'A', 23: '-', 24: '-', 25: 'A', 26: 'T', 27: 'A', 32: 'R', 33: 'A', 34: 'C', 35: 'T', 36: 'G', 37: 'T', 38: 'T', 39: 'C', 40: 'G', 43: 'A', 47: 'W', 52: 'C', 53: 'T', 54: 'A', 55: 'G', 57: 'M', 59: 'Y', 62: 'C', 63: 'T', 64: 'T', 65: 'A', 66: 'A', 68: 'A', 79: 'T', 83: 'C', 84: 'G', 87: 'G', 94: 'T', 96: 'T', 99: 'A', 100: 'T'}, 'Bos': {4: 'C', 10: 'W', 11: 'C', 12: 'A', 13: 'G', 14: 'A', 15: 'T', 17: 'A', 18: 'A', 19:

In [40]:
# maybe make genotype dict for each genus (could also make it string)
genotypes = {} # dict of lists: {Felis: ['T', 'T',...]}
for k,v in filtered_chars1.iteritems():
    for i in inform_alleles:
        geno = v[i]
        genotypes.setdefault(k, []).append(geno)
len(genotypes)
        

7

In [41]:
genotypes

{'Bos': ['C',
  'W',
  'C',
  'A',
  'G',
  'A',
  'T',
  'A',
  'A',
  'T',
  'T',
  'R',
  'Y',
  '-',
  'A',
  'T',
  'A',
  'A',
  'A',
  'A',
  'T',
  'T',
  'A',
  'T',
  'T',
  'Y',
  'G',
  'A',
  'T',
  'C',
  'Y',
  'A',
  'G',
  'A',
  'C',
  'C',
  'T',
  'T',
  'A',
  'A',
  'A',
  'T',
  'C',
  'G',
  'G',
  'T',
  'T',
  'N',
  'N'],
 'Canis': ['S',
  'A',
  'T',
  'A',
  'R',
  'R',
  'Y',
  'N',
  'R',
  'T',
  'Y',
  'Y',
  'W',
  '-',
  '-',
  'N',
  'Y',
  'A',
  'A',
  'A',
  'T',
  'A',
  'A',
  'T',
  'T',
  'C',
  'R',
  'A',
  'R',
  'Y',
  'T',
  'A',
  'G',
  'A',
  'Y',
  'C',
  'T',
  'T',
  'A',
  'A',
  'W',
  'K',
  'Y',
  'K',
  'K',
  'T',
  'Y',
  'C',
  'T'],
 'Capra': ['C',
  'A',
  'Y',
  'A',
  'A',
  'A',
  'T',
  'A',
  'A',
  'T',
  'T',
  'A',
  'C',
  'A',
  'A',
  'R',
  'A',
  'A',
  'R',
  'A',
  'T',
  'T',
  'A',
  'T',
  'T',
  'C',
  'G',
  'A',
  'T',
  'C',
  'C',
  'G',
  'R',
  'A',
  'C',
  'Y',
  'C',
  'C',
  'G',
  'A',
  'A',


### Need to add CIGAR functions to read score from usearch output, and assign bp of each seq